In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-test-augment/x_train_2.npy
/kaggle/input/train-test-augment/y_test_2.npy
/kaggle/input/train-test-augment/y_train_2.npy
/kaggle/input/train-test-augment/y_test_1.npy
/kaggle/input/train-test-augment/x_test_1.npy
/kaggle/input/train-test-augment/x_train_1.npy
/kaggle/input/train-test-augment/y_train_1.npy
/kaggle/input/train-test-augment/x_test_2.npy
/kaggle/input/mlsp-padded/reshaped/x_train548.npy
/kaggle/input/mlsp-padded/reshaped/x_train224.npy
/kaggle/input/mlsp-padded/reshaped/x_train768.npy
/kaggle/input/mlsp-padded/reshaped/x_train607.npy
/kaggle/input/mlsp-padded/reshaped/x_train1.npy
/kaggle/input/mlsp-padded/reshaped/x_train60.npy
/kaggle/input/mlsp-padded/reshaped/x_train945.npy
/kaggle/input/mlsp-padded/reshaped/x_train158.npy
/kaggle/input/mlsp-padded/reshaped/x_train706.npy
/kaggle/input/mlsp-padded/reshaped/x_train428.npy
/kaggle/input/mlsp-padded/reshaped/x_train229.npy
/kaggle/input/mlsp-padded/reshaped/x_train305.npy
/kaggle/input/mlsp-padded/resha

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
import json

In [3]:
with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
    dict1 = json.loads(f.read())
x_train =np.load("../input/train-test-augment/x_train_1.npy")
y_train = np.load("../input/train-test-augment/y_train_1.npy")
x_test = np.load("../input/train-test-augment/x_test_1.npy")
y_test = np.load("../input/train-test-augment/y_test_1.npy")

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(800, 128, 2584, 1)
(800, 10)
(200, 128, 2584, 1)
(200, 10)


In [5]:
x_train = np.vstack((x_train,np.load("../input/train-test-augment/x_train_2.npy")))
y_train = np.vstack((y_train,np.load("../input/train-test-augment/y_train_2.npy")))
x_test = np.vstack((x_test,np.load("../input/train-test-augment/x_test_2.npy")))
y_test = np.vstack((y_test,np.load("../input/train-test-augment/y_test_2.npy")))

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1600, 128, 2584, 1)
(1600, 10)
(400, 128, 2584, 1)
(400, 10)


In [7]:
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)

In [8]:
print(x_test.shape)
print(y_train.shape)
print(dict1)

(400, 128, 2584, 1)
(1600, 10)
{'Bark': 0, 'Meow': 1, 'Siren': 2, 'Shatter': 3, 'Knock': 4, 'Crying_and_sobbing': 5, 'Microwave_oven': 6, 'Vehicle_horn_and_car_horn_and_honking': 7, 'Doorbell': 8, 'Walk_and_footsteps': 9}


In [9]:
(temp, f, t, num) = x_train.shape
print(f)
print(t)
print(num)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

128
2584
1
(1600, 128, 2584, 1)
(400, 128, 2584, 1)
(1600, 10)
(400, 10)


In [10]:
N_classes = 10
def make_model(f, t, num):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(f, t, num)))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='leaky_relu'))
    #model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='leaky_relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_classes, activation='softmax'))
    return model
model = make_model(f, t, num)

2022-10-12 09:52:44.441698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 09:52:44.540019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 09:52:44.540784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 09:52:44.542500: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 1292, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 1292, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 646, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 646, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 646, 64)       18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 646, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 323, 64)       0

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

# Train model for 50 epochs, capture the history
history = model.fit(x_train, y_train, verbose = 1, epochs=40, validation_data=(x_test, y_test))

2022-10-12 09:53:23.100928: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2116812800 exceeds 10% of free system memory.
2022-10-12 09:53:25.464609: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2116812800 exceeds 10% of free system memory.
2022-10-12 09:53:27.143224: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-10-12 09:53:29.225826: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


50/50 [==============================] - ETA: 0s - loss: 2.4430 - precision: 0.3846 - recall: 0.0750

2022-10-12 09:53:39.724456: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 529203200 exceeds 10% of free system memory.
2022-10-12 09:53:40.576209: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 529203200 exceeds 10% of free system memory.


50/50 [==============================] - 15s 134ms/step - loss: 2.4430 - precision: 0.3846 - recall: 0.0750 - val_loss: 2.0633 - val_precision: 0.9412 - val_recall: 0.0400
Epoch 2/40
50/50 [==============================] - 4s 86ms/step - loss: 1.7540 - precision: 0.6770 - recall: 0.1781 - val_loss: 2.0117 - val_precision: 1.0000 - val_recall: 0.0250
Epoch 3/40
50/50 [==============================] - 4s 86ms/step - loss: 1.3299 - precision: 0.7976 - recall: 0.3275 - val_loss: 2.0054 - val_precision: 1.0000 - val_recall: 0.0050
Epoch 4/40
50/50 [==============================] - 4s 87ms/step - loss: 0.8795 - precision: 0.8968 - recall: 0.5650 - val_loss: 1.9638 - val_precision: 0.9583 - val_recall: 0.0575
Epoch 5/40
50/50 [==============================] - 4s 86ms/step - loss: 0.5654 - precision: 0.9241 - recall: 0.7150 - val_loss: 1.7014 - val_precision: 0.9811 - val_recall: 0.1300
Epoch 6/40
50/50 [==============================] - 4s 89ms/step - loss: 0.3851 - precision: 0.9535 - re

In [13]:
del(x_train)

In [20]:
model.save('model_5.h5')

In [14]:
yhat_probs = model.predict(x_test, verbose=1)
# predict crisp classes for test set
#yhat_classes = model.predict_classes(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
y_target = []
for m in y_test:
    r = np.where(m == 1)
    y_target.append(int(r[0]))
    #y_target.append()

2022-10-12 09:57:29.262636: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 529203200 exceeds 10% of free system memory.


13/13 [==============================] - 1s 37ms/step
[4 3 7 9 5 5 2 4 5 6 9 2 8 3 6 5 5 6 6 5 1 2 5 4 5 2 1 7 1 4 7 5 4 4 6 9 9
 7 0 0 6 4 8 6 3 0 2 3 1 6 7 3 2 8 3 3 6 1 4 0 4 7 5 4 9 8 0 8 3 7 5 6 5 2
 8 3 4 6 6 3 7 5 2 4 5 4 7 6 6 8 9 2 5 8 5 0 5 1 4 6 4 0 4 3 2 0 0 4 1 7 6
 6 0 5 8 5 1 8 9 1 2 1 7 3 0 0 7 2 0 0 9 2 7 6 5 2 6 2 9 5 1 8 1 1 8 7 8 4
 4 4 9 2 2 2 1 7 9 3 4 4 2 8 9 8 2 1 9 0 7 2 8 9 5 5 3 5 9 4 1 7 2 0 8 2 3
 7 2 5 4 7 7 1 7 7 1 7 8 7 1 2 6 7 9 4 0 9 3 4 4 8 3 2 3 4 6 0 0 4 1 3 1 9
 5 3 1 9 7 7 8 9 0 5 0 4 8 4 0 2 2 5 0 4 8 8 7 1 8 5 4 7 5 7 3 9 8 7 9 7 0
 0 6 9 2 0 9 7 6 4 1 2 7 9 4 4 4 0 8 1 0 1 3 8 9 3 4 0 5 9 3 3 5 2 8 4 2 6
 8 2 9 9 5 7 8 8 2 3 0 5 7 6 8 9 3 8 6 4 2 8 1 0 1 5 7 5 0 0 7 4 8 3 6 6 4
 7 1 1 5 4 1 8 5 3 5 7 1 4 3 7 6 7 1 0 5 0 5 1 3 4 6 2 1 5 4 7 4 4 4 8 0 8
 4 7 3 7 3 8 6 4 7 3 4 9 9 7 4 5 4 9 1 6 5 0 3 2 4 9 6 8 3 4]


In [15]:
print(len(y_hat))
print(len(y_target))

400
400


In [16]:
from sklearn.metrics import classification_report
report = classification_report(y_target, y_hat, output_dict=True)
#report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [17]:
lst = []
for m in range(10):
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)
lst.append("accuracy")
lst.append("macro avg")
lst.append("weighted avg")
report_df["Labels"] = lst

In [18]:
report_df

,precision,recall,f1-score,support,Labels
0,0.891892,0.8250,0.857143,40.0000,Bark
1,0.805556,0.7250,0.763158,40.0000,Meow
2,0.918919,0.8500,0.883117,40.0000,Siren
3,0.722222,0.6500,0.684211,40.0000,Shatter
4,0.660714,0.9250,0.770833,40.0000,Knock
5,0.818182,0.9000,0.857143,40.0000,Crying_and_sobbing
6,0.939394,0.7750,0.849315,40.0000,Microwave_oven
7,0.765957,0.9000,0.827586,40.0000,Vehicle_horn_and_car_horn_and_honking
8,0.950000,0.9500,0.950000,40.0000,Doorbell
9,0.735294,0.6250,0.675676,40.0000,Walk_and_footsteps


In [19]:
report_df.to_csv("model_5_result.csv")